<a href="https://colab.research.google.com/github/boyoungbk/sentiment_analysis-for_students-/blob/main/%EC%84%B1%EC%B0%B0%EC%9D%BC%EC%A7%80_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D(1)_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**성찰일지 긍정+부정 분석**

## AI교육방법 및 교육공학 세미나 리드 - AI 평가
발표일: 2020. 11. 18.(수)


```
<< 런타임 실행 전 >>
1. Colab에서 런타임 - 런타임 유형 변경 - 하드웨어 가속기를 GPU로 선택

```



# 1. 구글 드라이브와 Colab 연동
* [코랩(Colab)에서 구글 드라이브 마운트하여 데이터 분석에 활용하기](
https://m.blog.naver.com/chogar/221667083331)

In [ ]:
import os

In [20]:
# 구글 drive와 Colab 연동 (코드 실행 후 링크 클릭하고 인증키 입력하세요.)
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [25]:
# 구글 드라이브 경로 지정
os.listdir('/content/gdrive/My Drive/Colab Notebooks/sentiment_analysis')

['성찰일지 감성분석(1) - 형태소 분석.ipynb',
 'txt_folder',
 'txt_nouns',
 '(성찰수업) 나를 표현하는 키워드.gform',
 '성찰일지 감성분석(2) - 감성분석과 시각화.ipynb',
 '성찰일지 감성분석(2) - 감성사전 분석 후 시각화.ipynb']

In [ ]:
# 필요한 폴더 만들기

## ※ 한글 폰트 설치하기 (파이썬으로 시각화 하실 분 사용하세요)
* [구글 코랩(colab) 한글 깨짐 현상 해결방법](https://teddylee777.github.io/colab/colab-korean)
1. STEP 1. 나눔 폰트 설치 (Nanum)
2. STEP 2. 코랩(Colab)의 런타임을 재시작 합니다. == [런타임]-[런타임 다시 시작]
3. STEP 3. matplotlib의 폰트를 Nanum 폰트로 지정


In [ ]:
# # 나눔 폰트 설치 (Nanum)
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

# # STEP 3. matplotlib의 폰트를 Nanum 폰트로 지정
# import matplotlib.pyplot as plt
# plt.rc('font', family='NanumBarunGothic') 

# 2. 구글 스프레드시트 파일 불러오기
* [외부 데이터: 로컬 파일, 드라이브, 스프레드시트, Cloud Storage](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=qzi9VsEqzI-o)
* [Google Spreadsheets Python API v4
](https://github.com/burnash/gspread#more-examples)

In [ ]:
!pip install gspread

In [ ]:
# colab에 권한부여하고 gspread 라이브러리를 사용해 Google 스프레드시트와 상호작용
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
# 스프레드시트를 데이터프레임 pandas로 불러오기
import pandas as pd

**txt_folder의 스프레드 시트 파일 주소 수정하세요!**

In [ ]:
# 스프레드시트 파일 주소 입력
sht = gc.open_by_url('https://docs.google.com/spreadsheets/d/1-yiaahewjYvDwEGGKTOk2DgcCv97FS5rtE-m6ADWNo4/edit#gid=2020793792')

# 첫번째 시트 불러오기
sheet = sht.get_worksheet(0)
df = pd.DataFrame(sheet.get_all_records()) 
df.head()

# 모든 row를 list로 불러오기
# txt_data = worksheet.get_all_values()

# 3. 형태소 분석기로 텍스트 데이터 토큰화하기
## 형태소 분석기 사용방법 (KoNLPy의 Mecab)
* [KoNLPy](https://github.com/konlpy/konlpy)
* [Google Colab에서 Mecab-ko-dic 쉽게 사용하기](https://somjang.tistory.com/entry/Google-Colab%EC%97%90%EC%84%9C-Mecab-koMecab-ko-dic-%EC%89%BD%EA%B2%8C-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0)
* [Google Colab에서 mecab-ko-dic 사용자 사전 추가하기](https://somjang.tistory.com/entry/Google-Colab%EC%97%90%EC%84%9C-mecab-ko-dic-%EC%82%AC%EC%9A%A9%EC%9E%90-%EC%82%AC%EC%A0%84-%EC%B6%94%EA%B0%80%ED%95%98%EA%B8%B0)
* [Mecab 품사 태그 분류](https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0)


Mecab 설치 이유

In [ ]:
# KoNLPy의 Mecab 설치
# 1. git clone으로 저장소 가져오기
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

# mecab 설치 폴더 선택하고 shell script 실행
os.chdir('Mecab-ko-for-Google-Colab')
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
# KoNLPy의 Mecab 형태소 분석기 import하고 텍스트데이터 토큰화하기
from konlpy.tag import Mecab
mecab = Mecab()

#텍스트에 포함되어 있는 특수 문자 제거
def clean_text(text):
  tidy_text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text) 
  return tidy_text


# 불용어 처리 (자유롭게 추가해주세요.)
stopwords = ['때문', '가지']

# 형태소분석 함수 (2글자 이상 단어 & 불용어 제외)
def tokenizer(text):
  nouns2 = []
  tidy_text = clean_text(text)
  nouns = mecab.nouns(text)
  for noun in nouns:
    if len(noun) > 1 and noun not in stopwords:
      nouns2.append(noun)
  return nouns2

In [ ]:
# 토큰화 예제
text = u'''AI교육공학과 교육방법 정말 유익하고 재미있습니다. ^^ 감사합니다.'''
tokenizer(text)

In [ ]:
# df 데이터프레임에 토큰화한 단어 리스트 추가
# 열 삭제 하고 싶으실 때 == del df['삭제하고 싶은 열 이름']
df['단어 리스트'] = df['성찰일지'].apply(tokenizer)
df.head()

In [ ]:
# 날짜 태깅
from datetime import date
today = date.today().isoformat()

In [ ]:
# 형태소 분석 프레임 저장
df.to_csv(f'/content/gdrive/My Drive/Colab Notebooks/sentiment_analysis/txt_nouns/({today})성찰일지 분석.csv')